In [2]:
import os
import xlsxwriter
import numpy as np
import pandas as pd
from pandas import isnull

In [3]:
def CreateFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
    return directory

def FindDel(S):
    """This function takes a the first string in a list,(S) that starts with ENG, it finds the nomancature 
    the file uses for the begining of each file, e.g ENG44007 or ENGINX00305258, it shall also work if more numbers
    are added or the name changes so long as ENG followed by anything other than "." or "_" or " " is still 
    the start of each file"""
    for i in S:
        if i[0:3] == 'ENG':
            n = 1
            for j in i:
                if j == '.':
                    break
                elif j == ' ':
                    break
                elif j == '_':
                    break
                else: 
                    n += 1 
            return n
        else:
            continue


def InitialRun(S):
    """This takes S a list of filenames and returns the first run index"""
    last_int_index = FindDel(S) -1
    first_int_index = 0 
    for i in S:                                                          
        if i[0:3] == 'ENG':
            for j in range(last_int_index):
                try:
                    int(i[j])
                    run_number = i[first_int_index:last_int_index]
                    break
                except:
                    first_int_index += 1
        else:
            continue
        break
    return((run_number))

def CycleRuns(S):
    """This takes a list of filenames and returns a list of all the runs in the file"""
    Start_run = int(InitialRun(S))
    Last_run = int(InitialRun(reversed(S)))
    Runs = range(Start_run,Last_run)
    return (list(Runs))

def RunIndex(s,n): #ENGINX00305258
    """s is a string in the for m of a filename from the enginx data and returns it's run number, n is the FindDel"""
    last_int_index = n-1
    first_int_index = 0
    for j in range(last_int_index):
        try:
            int(s[j])
            return s[first_int_index:last_int_index]
        except:
            first_int_index += 1

### Making spreadsheet of file names 

In [ ]:
directory = "D:\\"
array_names = []
file_years = []
long_array_names = []
for foldername in os.listdir(directory):
    if foldername[0:5] == 'cycle':
        n = FindDel(os.listdir("D:\\"+foldername))
        print(n)
        for filename in os.listdir("D:\\"+foldername):
            if filename[n:] in array_names: #stop repeated file names from entering 
                pass
            elif len(filename) < n+1: # Stop useless file names from entering (e.g copy)
                pass
            elif filename[n] in ("S", "s", "N", "n"):# Stop temporary files from entering (e.g .s01 or .n001)
                try:
                    int(filename[n+1]) # all temp files are 's' or 'n' followed by a int
                    temp_files.append(filename)
                except ValueError: 
                    array_names.append(filename[n:])
                    long_array_names.append(filename)
                    file_years.append(foldername)
            else: # most unique file names go here 
                array_names.append(filename[n:])
                long_array_names.append(filename)
                file_years.append(foldername)
                
#print(file_years)
#print(array_names)
#print(long_array_names)

workbook   = xlsxwriter.Workbook('Filenames.xlsx')

worksheet1 = workbook.add_worksheet()
worksheet2 = workbook.add_worksheet()

worksheet1.write_column('A1', array_names)
worksheet2.write_column('A1', long_array_names)
worksheet1.write_column('B1', file_years)
worksheet2.write_column('B1', file_years)
workbook.close()

### Extracting times

In [ ]:
directory = "D:\\"
time = np.array([])
Experiment_index = np.array([])
for foldername in os.listdir(directory):
    if foldername[0:5] == 'cycle':
        n = FindDel(os.listdir("D:\\"+foldername))
        print(n)
        for filename in os.listdir("D:\\"+foldername):
            if filename[-4:] == ".txt":
                file_path = directory+foldername+"\\"+filename
                df = pd.read_csv(file_path, sep = "\t", names = ["Date\time", "other"])
                li = df["Date\time"].values
                if os.stat(file_path).st_size == 0: # remove empty files
                    pass
                elif len(str(df.iloc[0].values[0])) < 18: # remove the files without time in the first colmn
                    print(filename)
                    pass
                elif li[0][10] =="T":
                    for i in range(len(li)): # fix the T issue in some times 
                        untdate = li[0].replace('T',' ')
                        li = np.append(li,[untdate])
                        li = np.delete(li,[0], axis = None)
                    time = np.concatenate((a,li),axis=None)
                    Experiment_index = np.append(Experiment_index,[filename[:n-1]])
                else: # Normal time columns
                    time = np.concatenate((a,li),axis=None)
                    Experiment_index = np.append(Experiment_index,[filename[:n-1]])
                    #print(a)
print(len(time))
print(time)
print(Experiment_index)
print(len(Experiment_index))

### Making a dictionary from spreadsheet of filenames

In [3]:
path = "C:\\Users\\zahaab\\ENGIN_X_DATA_CLEANING\\File_names_2013-19.xlsx"
def Translator(path):
    xls = pd.ExcelFile(path)
    df = xls.parse(sheet_name = "Sheet1")
    df = df[["Filename", "Column_name"]]
    df = df.fillna(0)
    true_name_translater = dict(zip(df["Filename"],df["Column_name"]))
    s = set(true_name_translater.values())
    col_names = list(s)
    col_names.remove(0)
    col_names.append("measured_temperature_U")
    col_names.append("unregistered_value")
    return true_name_translater, col_names

true_name_translater, col_names = Translator(path)

### Extracting data from raw files 

In [4]:
def AddRawData(run_number):
    file_path = output+"\\"+str(run_number)+".txt"
    output_file = open(file_path, "r")
    contents = output_file.read()
    split_contents = contents.split("kamehameha") # Had to pick somthing that wouldn't be in data
    del split_contents[-1]
    clean_split_contents = []
    for i,j in zip(split_contents, data):
        a = i.replace("\n","").replace("[","").replace("]","")
        a = a.split(" ")
        a = list(filter(None, a))
        if "data" in j:
            clean_split_contents.append(j[0:16]+str(run_number))
            Neutron_file = open(Neutron_folder+"\\"+j[0:16]+str(run_number)+".txt", "w+")
            Neutron_file.write(str(a))
            Neutron_file.close()
        elif "time" in j:
            untdate = i.replace('T',' ')
            clean_split_contents.append(untdate)
        else:
            clean_split_contents.append(a)
    raw_data = dict(zip(data, list(clean_split_contents)))
    output_file.close()
    os.remove(file_path) 
    return raw_data

In [5]:
def NeutronDataTranslator(Neutron_key):
    Neutron_file = open(Neutron_folder+"\\"+Neutron_key+".txt", "r")
    contents = Neutron_file.read()
    contents = contents.replace("'","").replace("[","").replace("]","")
    contents = contents.split(", ")
    contents = list(map(float, contents))
    Neutron_file.close()
    return contents

In [ ]:
output= r"C:\Users\zahaab\temp_raw_data"
Neutron_folder = r"C:\Users\zahaab\Neutron_data"
data = ["North_neutron_X_data","North_neutron_Y_data","South_neutron_X_data",
            "South_neutron_Y_data","neutron_start_time","neutron_end_time","Run_Title"] #neutron data col names
raw_data = AddRawData(44296)
print(raw_data)
North_neutron_X_44296 = NeutronDataTranslator("North_neutron_X_44296")
print(North_neutron_X_44296)
print(type(North_neutron_X_44296))

### Making function(s) for extracting data from files 

In [4]:
def NewRun(run_number, run_data, time_list, raw_data, file_location):
    """This activates if a new run has started, it makes a new list of dictionaries and makes a CSV out of current data"""
    df1 = pd.DataFrame(columns = ["Date/Time"])
    for i in time_list:
        row = np.array([i])
        column_names = np.array(["Date/Time"])
        for j in run_data:
            try:
                row = np.append(row, [j[1][i]])
                column_names = np.append(column_names, [j[0]])
            except:
                pass
        df2 = pd.DataFrame([row], columns = column_names)
        df1 = df1.append(df2, sort=False)
    df1.set_index("Date/Time", drop=True)
    for i in raw_data:
        df1[str(i)] = str(raw_data[i])
    print(run_number)
    print(file_location)
    df1.to_csv(file_location+"\\"+run_number)
    
    raw_data = {}
    run_data = []
    time_list = np.array([])
    for i,j in enumerate(col_names):
        run_data.append((j,{}))
    
    return run_data,time_list,raw_data

def Find_dict(run_data, trans_filename):
    for i,j in enumerate(run_data):
        if j[0] == trans_filename:
            return i

def AddData(file_path, filename, n, run_data, time_list):
    """This takes data from a txt file and puts it into a list of times and the dictionaries"""
    df = pd.read_csv(file_path, sep = "\t", names = ["Date/Time", str(true_name_translater[filename[n:]])])
    file_times = df["Date/Time"].values
    file_data = df[str(true_name_translater[filename[n:]])].values 
    dict_index = Find_dict(run_data, true_name_translater[filename[n:]])
    if os.stat(file_path).st_size == 0: # remove empty files
        pass
    elif len(str(df.iloc[0].values[0])) < 18: # remove the files without time in the first colmn
        pass
    elif file_times[0][10] =="T":
        for i in range(len(li)): # fix the T issue in some times 
            untdate = file_times[0].replace('T',' ')
            file_times = np.append(file_times,[untdate])
            file_times = np.delete(file_times,0, axis = None)
        time_list = np.unique(np.append(time_list, file_times))
        for i, j in zip(file_times,file_data):
            if "temperature" in true_name_translater[filename[n:]]:#Alot of tempreture files, may be some overlap in time
                try:
                    a = run_data[dict_index][1][i]
                    run_data[-2][1][i] = j
                except:
                    run_data[dict_index][1][i] = j
            else:#rest of the files 
                try:
                    a = run_data[dict_index][1][i]
                    run_data[-1][1][i] = [run_data[dict_index][0],j]
                except:
                    run_data[dict_index][1][i] = j
    else: # Normal time columns
        time_list = np.unique(np.append(time_list, file_times))
        for i, j in zip(file_times,file_data):
            if "temperature" in true_name_translater[filename[n:]]:
                try:
                    a = run_data[dict_index][1][i]
                    run_data[-2][1][i] = j
                except:
                    run_data[dict_index][1][i] = j
            else:
                try:
                    a = run_data[dict_index][1][i]
                    run_data[-1][1][i] = [run_data[dict_index][0],j]
                except:
                    run_data[dict_index][1][i] = j
    return run_data, time_list


In [5]:
%%timeit
%prun
direc = "E:\\"
output= r"E:\temp_raw_data"
Neutron_folder = r"E:\Neutron_data"
ENGINX_DATA = r"E:\clean_data"
data = ["North_neutron_X_data","North_neutron_Y_data","South_neutron_X_data",
            "South_neutron_Y_data","neutron_start_time","neutron_end_time","Run_Title"] #neutron data col names


for foldername in os.listdir(direc):
    if foldername[0:5] == 'cycle':
        if foldername != "cycle_04_1":#just for test
            break
        folder_path = direc+"\\"+foldername
        clean_data = CreateFolder(ENGINX_DATA+"\\"+foldername)
        print(clean_data)
        n = FindDel(os.listdir(folder_path))
        filenames = os.listdir(folder_path)
        raw_data = {}
        run_data = []
        for i,j in enumerate(col_names):
            run_data.append((j,{}))
        time_list = np.array([])
        finale_file = 0
        for k, filename in enumerate(filenames):
            print(filename)
            file_path = folder_path+'\\'+filename
            if "Copy" in filename:
                pass 
            elif RunIndex(filenames[k],n) != RunIndex(filenames[k-1],n):
                if len(time_list) == 0:
                    pass
                else:
                    run_data,time_list,raw_data = NewRun(RunIndex(filenames[k-1],n), run_data, time_list, raw_data, clean_data)
                    if filename[-3:].lower() == "raw":
                        raw_data = AddRawData(RunIndex(filenames[k],n))
                    elif filename[-3:] == "txt":
                        run_data,time_list = AddData(file_path, filename, n, run_data, time_list)
                    else:
                        pass
            else:
                if filename[-3:].lower() == "raw":
                    raw_data = AddRawData(RunIndex(filenames[k],n))
                elif filename[-3:] == "txt":
                    run_data,time_list = AddData(file_path, filename, n, run_data, time_list)
                else:
                    pass
            finale_file = filenames[k]
        NewRun(RunIndex(filenames[k],n), run_data, time_list, raw_data, clean_data)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'E:\\'

### Finding how to deal with Stress rig data (work in progress)


In [27]:
from datetime import datetime

def CombineRigCleanData(cycle):
    direct = "E:\\stuff\\OtherData\\"+cycle+"\\Stress Rig"
    
    df1 = pd.DataFrame()
    for filename in os.listdir(ENGINX_DATA+"\\"+cycle):
        file_path = ENGINX_DATA+"\\"+cycle+"\\"+filename
        df2 = pd.read_csv(file_path,index_col=0)
        df2["Run_Number"] = filename
        df1 = df1.append(df2, ignore_index=False, sort=False)
    time_list = df1["Date/Time"].values
    df1.set_index("Date/Time") 

    df3 = pd.DataFrame()
    for filename in os.listdir(direct):
        file_path = direct+"\\"+filename
        df4 = pd.read_csv(file_path, sep = "|", skipinitialspace = True, skiprows = [0,1])
        df3 = df3.append(df4, ignore_index=False, sort=False)
        input_file = open(file_path+".txt", "r")
        first_line = output_file.readline()
        constants = {"Cross_Sectional_Area" : first_line[:33].split("=")[-1].replace(" ", ""),
                     "Gauge_Length_for_Strain1" : first_line[33:].split("=")[-1].replace(" ", "")}
        input_file.close()
        for i in constants:
            df3[str(i)] = str(constants[i]).replace("\n","")

    df3 = df3.rename(index = str, columns = dict(zip(
        df3.columns.values.tolist(), ['rig_{0}'.format(i).replace(" ", "") for i in df3.columns.values.tolist()])))
    df3 = df3.rename(index = str, columns = {'rig_Date/Time' : 'Date/Time'})

    #df3.set_index("Date/Time")
    #df3 = df3.loc[~df3.index.duplicated(keep='first')] #drop duplicates 
    df3 = df3.loc[~df3["Date/Time"].duplicated(keep='first')]
    df5 = df3.loc[df3["Date/Time"].isin(time_list)]
    result = pd.concat([df5, df1], axis=1, sort=False, join='outer')

    return(result) 

def CleanInstron01(cycle, filenames, clean_file_time):
    direct = r"E:\stuff\OtherData\INSTRON_01"
    
    df1 = pd.DataFrame()
    for filename in os.listdir(ENGINX_DATA+"\\"+cycle):
        file_path = ENGINX_DATA+"\\"+cycle+"\\"+filename
        df2 = pd.read_csv(file_path,index_col=0)
        df2["Run_Number"] = filename
        df1 = df1.append(df2, ignore_index=False, sort=False)
    time_list = df1["Date/Time"].values
    df1.set_index("Date/Time") 
    
    #find the files that needed for the spesific cycle
    cycle_start, cycle_end = time_list[0], time_list[-1]
    start_file_index, end_file_index = 0, 0
    for i in clean_file_time:
        if datetime.strptime(i, "%Y-%m-%d %H:%M:%S") > datetime.strptime(cycle_start, "%Y-%m-%d %H:%M:%S"):
            start_file_index = clean_file_time.index(i)-1
            break 
    for i in clean_file_time.reverse():
        if datetime.strptime(i, "%Y-%m-%d %H:%M:%S") < datetime.strptime(cycle_start, "%Y-%m-%d %H:%M:%S")::
            end_file_index = (clean_file_time.index(i)+1)*-1
            break 
    
    if start_file_index, end_file_index == 0,0:#Logic needs to be fixed
        return(df1)
    
    df3 = pd.DataFrame()
    for filename in os.listdir(direct)[start_file_index, end_file_index+1]:
        file_path = direct+"\\"+filename
        df4 = pd.read_csv(file_path, sep = "\t", skipinitialspace = True, skiprows = [0,1,2,3])
        df3 = df3.append(df4, ignore_index=False, sort=False)
        input_file = open(file_path+".txt", "r")
        first_line, second_line, third_line = output_file.readline(), output_file.readline(1), output_file.readline(2)
        constants = {"Cross_Sectional_Area" : first_line.split("=")[-1].replace(" ", ""),
                     "Gauge_Length_for_Strain1" : second_line.split("=")[-1].replace(" ", ""),
                     "RB Number" : third_line.split("=")[-1].replace(" ", "")}
        input_file.close()
        for i in constants:
            df3[str(i)] = str(constants[i]).replace("\n","")
        
    df3 = df3.rename(index = str, columns = dict(zip(
        df3.columns.values.tolist(), ['rig_{0}'.format(i).replace(" ", "") for i in df3.columns.values.tolist()])))
    df3 = df3.rename(index = str, columns = {'rig_Date/Time' : 'Date/Time'})
    df3 = df3.loc[~df3["Date/Time"].duplicated(keep='first')]
    
    unttime = []
    for i in df3["Date/Time"].values:
        untdate = "{}-{}-{} {}:{}:{}".format(
            date[0:4], date[5:7], date[8:10], date[11:13], date[14:16], date[17:19])
        unttime.append(untdate)
    df3 = df3.drop(columns=["Date/Time"])
    df3["Date/Time"] = unttime
    
    df5 = df3.loc[df3["Date/Time"].isin(time_list)]
    result = pd.concat([df5, df1], axis=1, sort=False, join='outer')
    return(result) 

In [26]:
ENGINX_DATA = r"E:\clean_data"

instron_start = len(os.listdir(ENGINX_DATA))

filenames = os.listdir(r"E:\stuff\OtherData\INSTRON_01")
clean_file_time = []
for filename in filenames:
    if "continuous" in filename:
        filenames.remove(filename)
    else: 
        date = filename[11:]
        untdate = "{}-{}-{} {}:{}:{}".format(
        date[0:4], date[5:7], date[8:10], date[11:13], date[14:16], date[17:19])
        clean_file_time.append(untdate)
            
for i,j in  enumerate(os.listdir(ENGINX_DATA)):
    if j == "cycle_17_3":
        CombineRigCleanData(j)
        instron_start = i
    elif i > instron_start:
        CleanInstron01(j, filenames, clean_file_time)
    else:
        CombineRigCleanData(j)

KeyboardInterrupt: 